### step1 获取月时间和日期（202304）
### step2 获取文件中含有当月日期的excel
### step3 读取该文件到pandas

In [1]:
import pandas as pd
import os
import datetime


In [2]:
current_date = datetime.datetime.now().strftime('%Y%m')  # 获取当前年月格式2023xx
# monthfirstday = datetime.datetime.now().strftime('%Y%m%d')
# monthfirstday
today = datetime.datetime.today()
# monthfirstday = datetime.datetime(today.year, today.month, 1, 0, 0, 0).strftime('%Y%m%d %H:%M:%S')


In [3]:
path1 = r"C:\Users\hank\PythonCode\Monthlybill"
files = os.listdir(path1)
pattern = 'CHW RTU_Monthly Report_'+current_date
for f in files:
    if pattern in f:
        path = f
        print(path)
# 模糊查找，文件夹下含有'CHW RTU_Monthly Report_'+current_date的文件
filepath = path1+"\\"+path


CHW RTU_Monthly Report_20230701000906205.xlsx


In [4]:
df = pd.read_excel(filepath,
                   sheet_name='Report',
                   header=1,
                   parse_dates=['datetime']
                   )
df.columns
# x = [10,104]
# df.drop(df.columns[x],axis=1,inplace=True)
# D1_QI_1150_BTU


Index(['datetime', 'D1_QQ_1150_BTU', 'D2_QQ_1150_BTU', 'D3_QQ_1150_BTU',
       'D4_QQ_1150_BTU', 'D5_QQ_1150_BTU', 'D6_QQ_1150_BTU', 'D7_QQ_1150_BTU',
       'D8_QQ_1150_BTU', 'D1_FT_1110',
       ...
       'OF_L_LD2', 'OF_L_LD2_Counter', 'SA_H_LD1', 'SA_H_LD1_Counter',
       'SA_H_LD2', 'SA_H_LD2_Counter', 'SA_L_LD1', 'SA_L_LD1_Counter',
       'SA_L_LD2', 'SA_L_LD2_Counter'],
      dtype='object', length=105)

In [5]:
df_QI = df[['datetime', 'D1_QI_1150_BTU', 'D2_QI_1150_BTU', 'D3_QI_1150_BTU', 'D4_QI_1150_BTU',
            'D5_QI_1150_BTU', 'D6_QI_1150_BTU', 'D7_QI_1150_1_BTU', 'D8_QI_1150_2_BTU']]
df_QI.drop(df_QI.index[0], inplace=True)


C:\Users\hank\AppData\Local\Temp\ipykernel_12392\536041591.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_QI.drop(df_QI.index[0], inplace=True)


In [6]:
#获取每日最大负荷QI
df_max_daily_QI = df_QI.groupby(pd.Grouper(key='datetime', axis=0,
                         freq='D')).max()
# df_max_daily_QI.drop(df_max_daily_QI.tail(1))

In [7]:
df_max_daily_QI.drop(df_max_daily_QI.tail(1).index,inplace=True)#删除最后一行
df_max_daily_QI

,D1_QI_1150_BTU,D2_QI_1150_BTU,D3_QI_1150_BTU,D4_QI_1150_BTU,D5_QI_1150_BTU,D6_QI_1150_BTU,D7_QI_1150_1_BTU,D8_QI_1150_2_BTU
datetime,,,,,,,,
2023-06-01,6512.0,7472.0,1794.0,704.0,1376.0,587.0,480.0,115.0
2023-06-02,4242.0,4823.0,1544.0,912.0,1294.0,571.0,470.0,180.0
2023-06-03,3872.0,5438.0,1438.0,477.0,567.0,153.0,403.0,142.0
2023-06-04,3524.0,4572.0,1332.0,627.0,613.0,142.0,459.0,119.0
2023-06-05,3590.0,3358.0,1443.0,409.0,1014.0,409.0,287.0,95.0
2023-06-06,3689.0,5378.0,2033.0,577.0,1268.0,383.0,383.0,148.0
2023-06-07,3645.0,4181.0,1420.0,533.0,1204.0,472.0,366.0,132.0
2023-06-08,4801.0,4545.0,1826.0,656.0,1100.0,440.0,393.0,144.0
2023-06-09,5456.0,4763.0,1954.0,1080.0,1176.0,428.0,620.0,179.0


In [8]:
df.drop(df.columns[9:105], axis=1, inplace=True)  # 删除后面的列
df.head()


,datetime,D1_QQ_1150_BTU,D2_QQ_1150_BTU,D3_QQ_1150_BTU,D4_QQ_1150_BTU,D5_QQ_1150_BTU,D6_QQ_1150_BTU,D7_QQ_1150_BTU,D8_QQ_1150_BTU
0,2023-05-31 23:00:00,kWH,kWH,kWH,kWH,kWH,kWH,kWH,kWH
1,2023-06-01 00:00:00,57619096,48370588,24665650,6783337,10697706,2521647,6214886,1237753
2,2023-06-01 01:00:00,57619604,48370812,24667306,6783560,10698076,2521702,6215315,1237754
3,2023-06-01 02:00:00,57619952,48371000,24668390,6783694,10698438,2521721,6215669,1237754
4,2023-06-01 03:00:00,57620108,48371180,24669084,6783770,10698796,2521721,6215980,1237754


In [9]:
df.drop(df.index[0], inplace=True)  # 删除第一行
df.head()


,datetime,D1_QQ_1150_BTU,D2_QQ_1150_BTU,D3_QQ_1150_BTU,D4_QQ_1150_BTU,D5_QQ_1150_BTU,D6_QQ_1150_BTU,D7_QQ_1150_BTU,D8_QQ_1150_BTU
1,2023-06-01 00:00:00,57619096,48370588,24665650,6783337,10697706,2521647,6214886,1237753
2,2023-06-01 01:00:00,57619604,48370812,24667306,6783560,10698076,2521702,6215315,1237754
3,2023-06-01 02:00:00,57619952,48371000,24668390,6783694,10698438,2521721,6215669,1237754
4,2023-06-01 03:00:00,57620108,48371180,24669084,6783770,10698796,2521721,6215980,1237754
5,2023-06-01 04:00:00,57620392,48371240,24670202,6783879,10699176,2521721,6216434,1237754


In [10]:
# daymax = df.groupby(pd.Grouper(key='datetime', axis=0,
#                       freq='D')).max()
daymin = df.groupby(pd.Grouper(key='datetime', axis=0,
                               freq='D')).min()
daymin_s = daymin[:-1]
daymin_e = daymin[1:]
daymin_s.head()


,D1_QQ_1150_BTU,D2_QQ_1150_BTU,D3_QQ_1150_BTU,D4_QQ_1150_BTU,D5_QQ_1150_BTU,D6_QQ_1150_BTU,D7_QQ_1150_BTU,D8_QQ_1150_BTU
datetime,,,,,,,,
2023-06-01,57619096,48370588,24665650,6783337,10697706,2521647,6214886,1237753
2023-06-02,57685808,48449504,24696822,6791896,10715755,2526314,6223565,1239243
2023-06-03,57734248,48497432,24723812,6803102,10731241,2530534,6231367,1241481
2023-06-04,57781652,48552104,24747450,6809984,10740434,2531325,6238608,1242883
2023-06-05,57825916,48603524,24769694,6817707,10748440,2532061,6245949,1244382


In [11]:
data_0 = df[df["datetime"].dt.hour == 0]  # .dt获取每天某小时的数据
data_0_s = data_0[:-1]
data_0_e = data_0[1:]
data_0_e.reset_index(inplace=True, drop=True)
data_0_e.index = data_0_e.index + 1
data_0_s.reset_index(
    inplace=True, drop=True
)  # 为了方便用pandas的减法，需要将index重置，这里用到reset_index
# data_0_e=data_0_e.drop(columns=[Index])
# data_0_s=data_0_s.drop(columns=[Index])
data_0_s.index = data_0_s.index + 1
dayuse = (
    data_0_e
    - data_0_s
)
# dayuse.drop(dayuse['datetime'],axis=1,inplace=True)
dayuse.head()


,datetime,D1_QQ_1150_BTU,D2_QQ_1150_BTU,D3_QQ_1150_BTU,D4_QQ_1150_BTU,D5_QQ_1150_BTU,D6_QQ_1150_BTU,D7_QQ_1150_BTU,D8_QQ_1150_BTU
1,1 days,66712,78916,31172,8559,18049,4667,8679,1490
2,1 days,48440,47928,26990,11206,15486,4220,7802,2238
3,1 days,47404,54672,23638,6882,9193,791,7241,1402
4,1 days,44264,51420,22244,7723,8006,736,7341,1499
5,1 days,39740,36812,21970,5680,13381,3562,5178,1191


In [12]:
# dayuse.index = daymin_s.index

dayuse.reset_index(drop=True, inplace=True)
dayuse.drop(columns=['datetime'], axis=1, inplace=True)
dayuse.set_index(daymin_s.index, inplace=True, verify_integrity=True)



In [13]:
dayuse.tail()

,D1_QQ_1150_BTU,D2_QQ_1150_BTU,D3_QQ_1150_BTU,D4_QQ_1150_BTU,D5_QQ_1150_BTU,D6_QQ_1150_BTU,D7_QQ_1150_BTU,D8_QQ_1150_BTU
datetime,,,,,,,,
2023-06-26,79664,79356,48134,13351,24526,6110,11258,2403
2023-06-27,86320,83464,52908,13319,25446,6633,12481,2411
2023-06-28,87012,85328,54620,18210,25209,6736,12805,2554
2023-06-29,84384,79128,41796,12825,20586,5878,10815,2564
2023-06-30,84436,78460,38956,19033,20399,6341,11701,2790


In [26]:
writer = dayuse['D2_QQ_1150_BTU'].to_excel('new.xlsx',sheet_name='Sheet_name_1')
with pd.ExcelWriter('new.xlsx') as writer:  
    dayuse['D1_QQ_1150_BTU'].to_excel(writer, sheet_name='Sheet_name_1')
    dayuse['D2_QQ_1150_BTU'].to_excel(writer, sheet_name='Sheet_name_2')